In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

In [14]:
np.lexsort?

In [17]:
def front_pareto(population, evaluations):
    np = population.shape[0]
    domination_count = np.zeros(np)
    pareto_front = np.empty(np)
    dominatees = []
    f = []
    for i in xrange(np):
        dominatees.append(np.nonzero(np.all(evaluations > evaluations[comp], axis=1))[0])
        domination_count[i] = np.sum(np.all(evaluations < evaluations[comp], axis=1))
        if domination_count[i] == 0:
            pareto_front[i] = 1
            f.append(i)
    
    fr = 2
    while f:
        nxt = []
        for p in f:
            for d in dominatees[p]:
                domination_count[d] -= 1
                if domination_count[d] == 0:
                    pareto_front[d] = fr
                    nxt.append(d)
        fr += 1
        f = nxt
    return pareto_front

In [16]:
def NSGA2_sort(population, evaluations):
    np = population.shape[0]
    nf = evaluations.shape[1]
    pareto_front = front_pareto(population, evaluations)
    distances = np.zeros(np)
        
    for i in xrange(nf):
        idxs = np.argsort(evaluations[:, i])
        population = population[idxs]
        evaluations = evaluations[idxs]
        pareto_front = pareto_front[idxs]
        distances = distances[idxs]
        distances[0] = np.inf
        distances[np] = np.inf
        distances[1:np-1] += evaluations[2:, i] - evaluations[:-2, i]
    
    idxs = np.lexsort(pareto_front, distances)
    return population[idxs], evaluations[idxs]

In [32]:
def NSGA2(iterations, genome_length, population_size, offspring_size, evaluation_functions, ranges):
    
    log_populations = np.empty((iterations, population_size, genome_length))
    
    lower_bounds = ranges[:, 0]
    upper_bounds = ranges[:, 1]
    
    current_population_solutions = np.random.rand(population_size, genome_length) * (upper_bounds - lower_bounds) + lower_bounds
    current_population_scores = np.empty((population_size, len(evaluation_functions)))
    print current_population_scores.shape
    print evaluation_functions[0](current_population_solutions).shape
    print current_population_scores[:, 0].shape
    for f in xrange(len(evaluation_functions)):
        current_population_scores[:, f] = (evaluation_functions[f])(current_population_solutions)
    
    for i in tqdm_notebook(xrange(iterations)):
        # parent selection
        par1Idx = np.random.choice(population_size, offspring_size)
        par2Idx = np.random.choice(population_size, offspring_size)
        
        # crossover
        alphas = np.random.rand(offspring_size, genome_length)
        offspring_population_solutions = alphas * current_population_solutions[par1Idx] + (1 - alphas) * current_population_solutions[par2Idx]
        # mutation TODO
        
        np.clip(offspring_population_solutions, lower_bounds, upper_bounds, out=offspring_population_solutions)
        
        # evaluation of offspring
        offspring_population_scores = np.empty((offspring_size, len(evaluation_functions)))
        for f in xrange(len(evaluation_functions)):
            offspring_population_scores[:, f] = evaluation_functions[f](offspring_population_solutions)
        
        combined_solutions, combined_evaluations = NSGA2_sort(np.vstack([current_population_solutions, offspring_population_solutions]), np.vstack([current_population_scores, offspring_population_scores]))
        current_population_solutions = combined_solutions[:population_size]
        current_population_scores = combined_evaluations[:population_size]
        log_populations[i, :, :] = population[:, :]
        
    return log_populations
        

In [19]:
def graph_population(population, evaluation_functions):
    scores = np.empty((population.shape[0], len(evaluation_functions)))
    for f in xrange(len(evaluation_functions)):
        scores[:, f] = evaluation_functions[f](population)
    
    plt.plot(figsize=(18, 6))
    plt.scatter(scores, c = front_pareto(population, scores))
    plt.show()

## SCH problem

In [20]:
def SCH1(pop):
    return pop ** 2

def SCH2(pop):
    return (pop - 2) ** 2

In [33]:
pops = NSGA2(100, 1, 100, 100, np.array([SCH1, SCH2]), np.array([[-1000, 1000]]))

(100, 2)
(100, 1)
(100,)


ValueError: could not broadcast input array from shape (100,1) into shape (100)